### Note: Duplicate this notebook so you can run mutliple processes with different api keys concurrently

The following code cell ensures that the underlying `.py` file gets automatically reloaded after you save any changes.  

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd    
from model import initialize_model, call_llm
import time

together_token2 = "-" # The api key from your second together account
model_name = "meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"

/Users/ethanyuxin/Documents/World/nlp_group19/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

def preprocess(file_path):
    data = pd.read_json(path_or_buf=file_path, lines=True)
    ids = set()
    rows_to_be_dropped = []
    for i in range(len(data)):
        row = data.iloc[i]
        if row.unique_id in ids:
            rows_to_be_dropped.append(i)
        else:
            ids.add(row.unique_id)
    data = data.drop(rows_to_be_dropped)
    return data

In [5]:
data = preprocess("./data/train.jsonl")

In [6]:
first_partition = data[:1365]
second_partition = data[1365:2730]
third_partition = data[2730:4095]
fourth_partition = data[4095:5460]
fifth_partition = data[5460:6825]
sixth_partition = data[6825:]

In [ ]:
labels = []
reasonings = []
raw_output = []
ids = []
client, _ = initialize_model("", defined_api_key=together_token2)
last_time = time.time()

# TODO Change len(data) to len(your_partition)
for i in range(len(fourth_partition)):
    if i > 0 and i % 60 == 0:
        print(i)
        current_time = time.time()
        time.sleep(max(0, 63 - (current_time - last_time)))
        last_time = time.time()

    # TODO Change data to your partition
    current_data = fourth_partition.iloc[i]
    ids.append(current_data.unique_id)
    classification, reasoning = call_llm(client, model_name, current_data.string, current_data.sectionName)
    raw_output.append(classification)
    labels.append(classification)
    reasonings.append(reasoning)
    

id='noB7MrM-4Yz4kd-928d5ae80dabfdf6' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743397981 model='meta-llama/Llama-3.3-70B-Instruct-Turbo-Free' choices=[ChatCompletionChoicesData(index=0, logprobs=None, seed=12397685816600949000, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='{"classification": "method", "reasoning": "The text describes the specific procedures and criteria used to select and exclude participants from the study, such as excluding men with missing information, extreme blood pressure values, or a history of certain diseases. This level of detail about the study\'s methodology is typical of a methods section in a scientific paper."}', tool_calls=[]))] prompt=[] usage=UsageData(prompt_tokens=288, completion_tokens=71, total_tokens=359)
id='noB7NNS-3NKUce-928d5aeffe66fdf6' object=<ObjectType.ChatCompletion: 'chat.completion'> created=1743397983 model='meta-llama/Llama-3.3

In [ ]:
df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
df.to_csv("fourth_partition_ethan.csv")